In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import subprocess
from com.yahoo.ml.tf import TFCluster
import mnist_dist

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("-e", "--epochs", help="number of epochs", type=int, default=0)
parser.add_argument("-i", "--images", help="HDFS path to MNIST images in parallelized format")
parser.add_argument("-f", "--format", help="example format: (csv|pickle|tfr)", choices=["csv","pickle","tfr"], default="csv")
parser.add_argument("-m", "--model", help="HDFS path to save/load model during train/inference", default="mnist_model")
parser.add_argument("-o", "--output", help="HDFS path to save test/inference output", default="predictions")
parser.add_argument("-r", "--readers", help="number of reader/enqueue threads", type=int, default=1)
parser.add_argument("-s", "--steps", help="maximum number of steps", type=int, default=1000)
parser.add_argument("-X", "--mode", help="train|inference", default="train")
parser.add_argument("-c", "--rdma", help="use rdma connection", default=False)
parser.add_argument("-tb", "--tensorboard", help="launch tensorboard process", action="store_true")


_StoreTrueAction(option_strings=['-tb', '--tensorboard'], dest='tensorboard', nargs=0, const=True, default=False, type=None, choices=None, help='launch tensorboard process', metavar=None)

In [4]:
num_executors = 3
cluster = TFCluster.reserve(sc, num_executors, 1, True, TFCluster.InputMode.TENSORFLOW)

2017-02-03 02:19:13,676 INFO (MainThread-8373) Reserving TFSparkNodes w/ TensorBoard
2017-02-03 02:19:25,401 INFO (MainThread-8373) TensorBoard running at: http://ip-172-31-17-161:32879


{'addr': ('ip-172-31-30-39', 40213), 'task_index': 0, 'port': 47014, 'authkey': UUID('b4c07235-044a-4c2b-b3ce-405ff5a850b8'), 'worker_num': 0, 'host': 'ip-172-31-30-39', 'ppid': 28325, 'job_name': 'ps', 'tb_port': 0}
{'addr': '/tmp/pymp-uNclLs/listener-UQnli9', 'task_index': 0, 'port': 35767, 'authkey': UUID('9aa3fad5-83dc-4cc7-9967-f6c02bd10e1f'), 'worker_num': 1, 'host': 'ip-172-31-17-161', 'ppid': 2142, 'job_name': 'worker', 'tb_port': 32879}
{'addr': '/tmp/pymp-OaWzph/listener-_72wmx', 'task_index': 1, 'port': 60383, 'authkey': UUID('7daff8e0-4226-4c2e-9e28-a258cff4c054'), 'worker_num': 2, 'host': 'ip-172-31-27-176', 'ppid': 976, 'job_name': 'worker', 'tb_port': 0}


In [9]:
subprocess.call(["hadoop", "fs", "-rm", "-R", "/user/root/mnist_model"])

0

In [10]:
args = parser.parse_args(['--format', 'tfr', '--model', 'mnist_model', '--images', 'mnist/tfr/train'])
cluster.start(mnist_dist.map_fun, args)

2017-02-03 02:06:38,293 INFO (MainThread-7718) Starting TensorFlow


In [11]:
cluster.shutdown()

2017-02-03 02:06:48,118 INFO (MainThread-7718) Stopping TensorFlow nodes


connecting to ('ip-172-31-17-161', 41015), e0bf9da4-8d9a-4bf7-9dd3-5a12e7428bb7


In [12]:
print(subprocess.check_output(["hadoop", "fs", "-ls", "/user/root/mnist_model"]))

Found 17 items
-rw-r--r--   3 root supergroup        265 2017-02-03 02:08 /user/root/mnist_model/checkpoint
-rw-r--r--   3 root supergroup     142729 2017-02-03 02:06 /user/root/mnist_model/graph.pbtxt
-rw-r--r--   3 root supergroup     814164 2017-02-03 02:07 /user/root/mnist_model/model.ckpt-471.data-00000-of-00001
-rw-r--r--   3 root supergroup        372 2017-02-03 02:07 /user/root/mnist_model/model.ckpt-471.index
-rw-r--r--   3 root supergroup      56850 2017-02-03 02:07 /user/root/mnist_model/model.ckpt-471.meta
-rw-r--r--   3 root supergroup     814164 2017-02-03 02:08 /user/root/mnist_model/model.ckpt-595.data-00000-of-00001
-rw-r--r--   3 root supergroup        372 2017-02-03 02:08 /user/root/mnist_model/model.ckpt-595.index
-rw-r--r--   3 root supergroup      56850 2017-02-03 02:08 /user/root/mnist_model/model.ckpt-595.meta
-rw-r--r--   3 root supergroup     814164 2017-02-03 02:08 /user/root/mnist_model/model.ckpt-719.data-00000-of-00001
-rw-r--r--   3 root supergroup       

In [5]:
inf_args = parser.parse_args(['--format', 'tfr', '--model', 'mnist_model', '--mode', 'inference', '--images', 'mnist/tfr/test', '--epochs', '1'])

In [6]:
cluster.start(mnist_dist.map_fun, inf_args)

2017-02-03 02:19:49,536 INFO (MainThread-8373) Starting TensorFlow


In [7]:
cluster.shutdown()

2017-02-03 02:19:56,503 INFO (MainThread-8373) Stopping TensorFlow nodes


connecting to ('ip-172-31-30-39', 40213), b4c07235-044a-4c2b-b3ce-405ff5a850b8


In [11]:
predictions =sc.textFile("/user/root/predictions")

In [12]:
predictions.take(10)

[u'7 7',
 u'2 2',
 u'1 1',
 u'0 0',
 u'4 4',
 u'1 1',
 u'4 4',
 u'9 9',
 u'5 6',
 u'9 9']